In [12]:
import pandas as pd
from functions import epoch, epochs, cutDf
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [3]:
matches = pd.read_csv('../Data/matches_glicko2.csv')

In [4]:
matches['tourney_date'] = matches['tourney_date'].apply(
    lambda x: pd.to_datetime(str(x), format='%Y%m%d')) # convert the tourney_date to datetime

In [8]:
np.sum([1,2,3])

6

In [ ]:
start = time.time()
epochs(cutDf(matches_glicko2,756977,805977,'tourney_date'),365)
compute_time = time.time()-start
print(compute_time)

In [128]:
for x in []:
    print(type(x))

In [122]:
players_to_instantiate = list(set([1,2]) - set([1,2]))
new_players = {player: Player() for player in players_to_instantiate}

{1: <pyglicko2.glicko2.Player at 0x7fe0b0226c70>}

In [138]:
def epoch(matches, players_dict):
    '''Take in a dataframe with matches, a list of players, a dictionary of players with the elements
    of player list as keys and an instance of the glicko2 class Player() as the value.
    This function wll return the updated players_list and players_dict.'''
    
    players_list = list(players_dict.keys())
    players = [p for p in np.append(matches['winner_id'].unique(),\
                                                matches['loser_id'].unique())]
    # instantiate player class for players not yet instantiated
    players_to_instantiate = list(set(players) - set(players_list))
    
    new_players = {player: Player() for player in players_to_instantiate}
    # print(type(new_players))

    # update list of instantiated players
    players_list = list(set(players).union(set(players_list)))
    # update dictionary of instantiated players
    players_dict.update(new_players)
    # print(type(players_dict))
    # determine who competed and who didn't
    players_dnc = list(set(players_list) - set(players))
    players_compete = list(set(players_list)-set(players_dnc))
    # print(f'{players_compete=}')
    # update rating deviation for players who didn't compete
    for player in players_dnc:
        players_dict[player].did_not_compete()
    # fill dictionary with (wins,losses), a tuple of list of opponents in wins and losses for each match
    results = {}
    # update players who did compete
    for player in players_compete:
        # get id of players they beat
        wins = [winner for winner in matches[matches['winner_id']==player]['loser_id']]
        # get id of players they lost to
        losses = [loser for loser in matches[matches['loser_id']== player]['winner_id']]
        # get opponents' rating and rd
        losses_rating = [players_dict[loss].getRating() for loss in losses]
        losses_rd = [players_dict[loss].getRd() for loss in losses]
        wins_rating = [players_dict[win].getRating() for win in wins]
        wins_rd = [players_dict[win].getRd() for win in wins]
        outcomes = np.append(np.ones(len(wins)),np.zeros(len(losses)))
        opponent_rating = wins_rating + losses_rating
        # print(f'{opponent_rating=}')
        opponent_rd = wins_rd + losses_rd
        results[player] = (opponent_rating, opponent_rd, outcomes)
        
    # update players
    for player in list(results.keys()):
        (rating_list, RD_list, outcome_list) = results[player]
        # print(f'{rating_list=}, {RD_list=}, {outcome_list=}')
        players_dict[player].update_player(rating_list, RD_list, outcome_list)
    return players_dict
    # opponent_indices = matches[matches['loser_id'].str==player '']


In [140]:
epoch(matches[100: 200],{})

{114688: <pyglicko2.glicko2.Player at 0x7fe0b00cae80>,
 114689: <pyglicko2.glicko2.Player at 0x7fe0b00ca3d0>,
 114690: <pyglicko2.glicko2.Player at 0x7fe0b00cae20>,
 114692: <pyglicko2.glicko2.Player at 0x7fe0e15d4610>,
 114693: <pyglicko2.glicko2.Player at 0x7fe0e15d4640>,
 114695: <pyglicko2.glicko2.Player at 0x7fe0e15d46a0>,
 114697: <pyglicko2.glicko2.Player at 0x7fe0e15d4700>,
 114699: <pyglicko2.glicko2.Player at 0x7fe0e15d4760>,
 114700: <pyglicko2.glicko2.Player at 0x7fe0e15d47c0>,
 114701: <pyglicko2.glicko2.Player at 0x7fe0e15d4820>,
 114702: <pyglicko2.glicko2.Player at 0x7fe0e15d4880>,
 114703: <pyglicko2.glicko2.Player at 0x7fe0e15d48e0>,
 114704: <pyglicko2.glicko2.Player at 0x7fe0e15d4940>,
 114705: <pyglicko2.glicko2.Player at 0x7fe0e15d49a0>,
 114706: <pyglicko2.glicko2.Player at 0x7fe0e15d4a00>,
 114707: <pyglicko2.glicko2.Player at 0x7fe0e15d4a60>,
 114708: <pyglicko2.glicko2.Player at 0x7fe0e15d4ac0>,
 114709: <pyglicko2.glicko2.Player at 0x7fe0e15d4b20>,
 114710: <

In [24]:
matches.shape[0]/7000

125.28257142857143

In [102]:
matches[11800: 16997].shape

(5197, 4)

In [76]:
matches[11650:16997].shape

(5347, 4)

In [131]:
start = time.time()
epoch(matches[12837:17000],{})
compute_time = time.time()-start
print(compute_time)

KeyboardInterrupt: 

In [137]:
start = time.time()
epoch(matches[12837:14000],{})
compute_time = time.time()-start
print(compute_time)

0.21326708793640137


In [113]:
start = time.time()
epoch(matches[12900: 17000],{})
compute_time = time.time()-start
print(compute_time)


0.41458582878112793


In [4]:
[x for x in zip([1,2,3],[2,3,4])]

[(1, 2), (2, 3), (3, 4)]

In [5]:
def epochs(match_history, interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length
    
    Next iterations: generate a rating history indicating the rating of each player each time that
    the rating updates.
    '''
    # check if the match_history is empty, as if it is the function call will not complete.
    if match_history.empty:
        return "Try again with a non-empty match history!"
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range] # The times that
    # divide the matches into each epoch.
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1]) # each epoch will include matches
    # greater than or equal to the first element, less than the second element for the zip
    # generator's respective item.
    print([r for r in epoch_ranges])
    players_dict = {} # instantiate the dictionary that will hold a Player() class for each player.

    # iteratively re-update for each epoch
    for period in epoch_ranges:
        # rating_ period is the df of matches that fall within a given epoch
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        # If the rating period is empty, then adjust the rating deviation of the players.
        if not rating_period.empty:
            players_dict = epoch(rating_period,players_dict)
        else:
            for player in players_dict:
                players_dict[player].did_not_compete()
    # get the final rating period updates (for matches in the final 365 to 729 days).
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    players_dict = epoch(rating_period,players_dict)
    return players_dict

In [6]:
type(matches['tourney_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
epoch(matches[matches['tourney_date'] <= "1879-07-09"],{})

NameError: name 'np' is not defined

In [ ]:
matches[matches['tourney_date'] <= "1879-07-09"]